In [ ]:
import pandas as pd

# importando base de dados.
df = pd.read_csv('cancelamentos.csv')

# removendo colunas desnecessárias.
df.drop(columns=['CustomerID'], inplace=True)

# removendo linhas com dados em branco.
df.dropna(inplace=True)
#display(df.info())

In [ ]:
# removendo colunas com muitos dados diferentes, pois são mais fáceis de analisar com gráficos.
colmuns = list(df.columns)
for colmun in ['tempo_como_cliente','idade','frequencia_uso', 'dias_atraso', 'total_gasto','meses_ultima_interacao']:
    colmuns.remove(colmun)

# visualizando as quantidades e porcentagem das colunas da base de dados.
for colmun in colmuns:
    display(df[colmun].value_counts(normalize=True).map("{:.2%}".format))
    display(df[colmun].value_counts())
    

In [ ]:
# visualizando as quantidades e porcentagem das colunas da base de dados.
colmuns = list(df.columns)
colmuns.remove('cancelou')

for colmun in colmuns:
    display(df[colmun].value_counts(normalize=True).map("{:.2%}".format))
    display(df.groupby(colmun).mean(numeric_only=True))
    

In [ ]:
import plotly.express as px

for colmun in df.columns:
    graph = px.histogram(df, x = colmun, color="cancelou", width=600)
    graph.show()

In [ ]:
# Com os problemas identificados, aplicam-se as tratativas para comparar com o resultado inicial.

print('Estatisticas iniciais')
display(df['cancelou'].value_counts(normalize=True).map("{:.2%}".format))
display(df['cancelou'].value_counts())

print('Removendo os contratos mensais')
df_new = df[df['duracao_contrato'] != 'Monthly']
display(df_new['cancelou'].value_counts(normalize=True).map("{:.2%}".format))
display(df_new['cancelou'].value_counts())

print('Removendo resolvendo o problema antes de 5 ligacoes')
df_new = df[df['ligacoes_callcenter'] < 5]
display(df_new['cancelou'].value_counts(normalize=True).map("{:.2%}".format))
display(df_new['cancelou'].value_counts())

print('Nao permitindo mais de 20 dias de atraso no pagamento')
df_new = df[df['dias_atraso'] < 20]
display(df_new['cancelou'].value_counts(normalize=True).map("{:.2%}".format))
display(df_new['cancelou'].value_counts())

print('Aplicando todas ao memsmo tempo')
df_new = df[(df['duracao_contrato'] != 'Monthly') & (df['ligacoes_callcenter'].lt(5)) & (df['dias_atraso'].lt(20))]
display(df_new['cancelou'].value_counts(normalize=True).map("{:.2%}".format))
display(df_new['cancelou'].value_counts())

In [ ]:
# calculando a entropia e ganho de informação das colunas.
from collections import Counter
import math

def calculate_entropy(data):
    counter = Counter(data)
    total_samples = len(data)
    entropy = 0.0
    for count in counter.values():
        probability = count / total_samples
        entropy -= probability * math.log2(probability)
    return entropy

def calculate_information_gain(data, attribute, target):
    entropy_before_split = calculate_entropy(data[target])
    values = data[attribute].unique()
    total_samples = len(data)
    entropy_after_split = 0.0

    for value in values:
        subset = data[data[attribute] == value]
        weight = len(subset) / total_samples
        entropy_after_split += weight * calculate_entropy(subset[target])

    information_gain = entropy_before_split - entropy_after_split
    return information_gain

colmuns = list(df.columns)
colmuns.remove('cancelou')

print('Ganho de informacao da base de dados sem modificacao')
list_information_gain = []
for colmun in colmuns:
    dict_data = {colmun: calculate_information_gain(df, colmun, 'cancelou')}
    list_information_gain.append(dict_data)

list_information_gain = sorted(list_information_gain, key= lambda x: list(x.values())[0], reverse=True)
display(list_information_gain[:5])

print('Ganho de informacao da base de dados com as modificacoes')
list_information_gain = []
for colmun in colmuns:
    dict_data = {colmun: calculate_information_gain(df_new, colmun, 'cancelou')}
    list_information_gain.append(dict_data)

list_information_gain = sorted(list_information_gain, key= lambda x: list(x.values())[0], reverse=True)
display(list_information_gain[:5])
